In [3]:
import pandas as pd, numpy as np
import re

##### Load the data using read_csv function from pandas package

In [4]:
reviews0 = pd.read_csv("Zomato_reviews.csv")

In [5]:
reviews0.head()

,rating,review_text
0,1.0,"Their service is worst, pricing in menu is dif..."
1,5.0,really appreciate their quality and timing . I...
2,4.0,"Went there on a Friday night, the place was su..."
3,4.0,A very decent place serving good food.\r\nOrde...
4,5.0,One of the BEST places for steaks in the city....


In [6]:
reviews0.describe(include="all")

,rating,review_text
count,27762.000000,27748
unique,NaN,10548
top,NaN,good
freq,NaN,278
mean,3.665784,NaN
std,1.284573,NaN
min,1.000000,NaN
25%,3.000000,NaN
50%,4.000000,NaN
75%,5.000000,NaN


##### Null values in the review text? 

##### Remove the records where the review text is null

##### 14 rows are missing the review text - need to get rid of these records

In [7]:
reviews1 = reviews0[~reviews0.review_text.isnull()].copy()
reviews1.reset_index(inplace=True, drop=True)

In [8]:
reviews0.shape, reviews1.shape

((27762, 2), (27748, 2))

#### Converting to list for easy manipulation

In [9]:
reviews_list = reviews1.review_text.values

In [10]:
len(reviews_list)

27748

### Text clean up 
- Normalize the case  
- Remove stop words
   - remove "not", "no" from the stop word list
- Remove punctuations

##### Normalizing case

In [11]:
reviews_lower = [txt.lower() for txt in reviews_list]

In [12]:
reviews_lower[2:4]

['went there on a friday night, the place was surprisingly empty. interesting menu which is almost fully made of dosas. i had bullseye dosa and cheese masala dosa. the bullseye dosa was really good, with the egg perfectly cooked to a half boiled state. the masala in the cheese masala was good, but the cheese was a bit too chewy for my liking. the chutney was good, the sambar was average. the dishes are reasonably priced.',
 'a very decent place serving good food.\r\nordered chilli fish, chicken & pork sizzler.\r\neverything tasted good but pork could have been slightly better cooked.\r\ntried 2 beverages, both were very sweet.']

##### Remove extra line breaks

In [13]:
reviews_lower = [" ".join(txt.split()) for txt in reviews_lower]

In [14]:
reviews_lower[2:4]

['went there on a friday night, the place was surprisingly empty. interesting menu which is almost fully made of dosas. i had bullseye dosa and cheese masala dosa. the bullseye dosa was really good, with the egg perfectly cooked to a half boiled state. the masala in the cheese masala was good, but the cheese was a bit too chewy for my liking. the chutney was good, the sambar was average. the dishes are reasonably priced.',
 'a very decent place serving good food. ordered chilli fish, chicken & pork sizzler. everything tasted good but pork could have been slightly better cooked. tried 2 beverages, both were very sweet.']

#### Tokenize

In [15]:
from nltk.tokenize import word_tokenize

In [16]:
print(word_tokenize(reviews_lower[0]))

['their', 'service', 'is', 'worst', ',', 'pricing', 'in', 'menu', 'is', 'different', 'from', 'bill', '.', 'they', 'can', 'give', 'you', 'a', 'bill', 'with', 'increased', 'pricing', '.', 'even', 'for', 'serving', 'water', ',', 'menu', ',', 'order', 'you', 'need', 'to', 'call', 'them', '3-4', 'times', 'even', 'on', 'a', 'non', 'busy', 'day', '.']


In [17]:
reviews_tokens = [word_tokenize(sent) for sent in reviews_lower]
print(reviews_tokens[0])

['their', 'service', 'is', 'worst', ',', 'pricing', 'in', 'menu', 'is', 'different', 'from', 'bill', '.', 'they', 'can', 'give', 'you', 'a', 'bill', 'with', 'increased', 'pricing', '.', 'even', 'for', 'serving', 'water', ',', 'menu', ',', 'order', 'you', 'need', 'to', 'call', 'them', '3-4', 'times', 'even', 'on', 'a', 'non', 'busy', 'day', '.']


### Remove stop words and punctuations

In [18]:
from nltk.corpus import stopwords
from string import punctuation

In [19]:
stop_nltk = stopwords.words("english")
stop_punct = list(punctuation)

In [20]:
print(stop_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Note: Terms like ‘no’, ‘not’, ‘don’, ‘won’ are important, don’t remove

In [21]:
stop_nltk.remove("no")
stop_nltk.remove("not")
stop_nltk.remove("don")
stop_nltk.remove("won")

In [22]:
"no" in stop_nltk

False

In [23]:
stop_final = stop_nltk + stop_punct + ["...", "``","''", "====", "must"]

In [24]:
def del_stop(sent):
    return [term for term in sent if term not in stop_final]

In [25]:
del_stop(reviews_tokens[1])

['really',
 'appreciate',
 'quality',
 'timing',
 'tried',
 'thattil',
 'kutti',
 'dosa',
 "'ve",
 'addicted',
 'dosa',
 'really',
 'chutney',
 'really',
 'good',
 'money',
 'worth',
 'much',
 'better',
 'thattukada',
 'try']

In [26]:
reviews_clean = [del_stop(sent) for sent in reviews_tokens]

In [27]:
reviews_clean = [" ".join(sent) for sent in reviews_clean]
reviews_clean[:2]

['service worst pricing menu different bill give bill increased pricing even serving water menu order need call 3-4 times even non busy day',
 "really appreciate quality timing tried thattil kutti dosa 've addicted dosa really chutney really good money worth much better thattukada try"]

### Separate X and Y and perform train test split, 70-30

In [28]:
len(reviews_clean)

27748

In [29]:
X = reviews_clean
y = reviews1.rating

Train test split

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

### Document term matrix using TfIdf

* Use TF-IDF values for the terms as features to get into a vector space model

* Import TF-IDF vectorizer from sklearn

* Instantiate with a maximum of 5000 terms in your vocabulary

* Fit and apply on the train set

* Apply on the test set

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
vectorizer = TfidfVectorizer(max_features = 5000)

In [37]:
len(X_train), len(X_test)

(19423, 8325)

In [38]:
X_train_bow = vectorizer.fit_transform(X_train)

In [39]:
X_test_bow = vectorizer.transform(X_test)

In [40]:
X_train_bow.shape, X_test_bow.shape

((19423, 5000), (8325, 5000))

### Model building

* Model building: Random Forest Regressor

* Instantiate RandomForestRegressor from sklearn (set random seed)

* Fit on the train data

* Make predictions for the train set

In [43]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [44]:
?RandomForestRegressor

Init signature:
RandomForestRegressor(
    n_estimators=100,
    *,
    criterion='mse',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest regressor.

A random forest is a meta estimator that fits a number of classifying
decision trees on various sub-samples of the dataset and uses averaging
to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the fore

In [46]:
learner_rf = RandomForestRegressor(random_state=42)

In [47]:
%%time
learner_rf.fit(X_train_bow, y_train)

Wall time: 9min


RandomForestRegressor(random_state=42)

In [48]:
y_train_preds = learner_rf.predict(X_train_bow)

Model evaluation

Report the root mean square error

In [50]:
from sklearn.metrics import mean_squared_error

In [52]:
mean_squared_error(y_train, y_train_preds)**0.5

0.23720347586757728

#### Increase the number of trees

In [53]:
learner_rf = RandomForestRegressor(random_state=42, n_estimators=20)

In [54]:
%%time
learner_rf.fit(X_train_bow, y_train)

Wall time: 1min 37s


RandomForestRegressor(n_estimators=20, random_state=42)

In [55]:
y_train_preds = learner_rf.predict(X_train_bow)

In [56]:
mean_squared_error(y_train, y_train_preds)**0.5

0.2507584827360229

### Hyper-parameter tuning

* Hyperparameter tuning

* Import GridSearch

* Provide the parameter grid to choose:

* max_features – ‘auto’, ‘sqrt’, ‘log2’

* max_depth – 10, 15, 20, 25

Let's find the best hyper-parameters for the RandomForest Regressor classifier

In [57]:
from sklearn.model_selection import GridSearchCV

In [58]:
?RandomForestRegressor

Init signature:
RandomForestRegressor(
    n_estimators=100,
    *,
    criterion='mse',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest regressor.

A random forest is a meta estimator that fits a number of classifying
decision trees on various sub-samples of the dataset and uses averaging
to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the fore

Instantiate the learner with a random state

In [59]:
learner_rf = RandomForestRegressor(random_state=42)

In [60]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_features': [500, "sqrt", "log2", "auto"],
    'max_depth': [10, 15, 20, 25]
}

In [62]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = learner_rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 1, scoring = "neg_mean_squared_error" )

In [63]:
grid_search.fit(X_train_bow, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 15, 20, 25],
                         'max_features': [500, 'sqrt', 'log2', 'auto']},
             scoring='neg_mean_squared_error', verbose=1)

In [68]:
grid_search.best_score_

-0.45644900985829145

In [67]:
grid_search.best_estimator_

RandomForestRegressor(max_depth=25, max_features=500, random_state=42)

### Using the best estimator to make predictions on the test set

In [69]:
y_train_pred = grid_search.best_estimator_.predict(X_train_bow)

In [71]:
y_test_pred = grid_search.best_estimator_.predict(X_test_bow)

What is the root mean squared error on the test set?



In [72]:
mean_squared_error(y_train, y_train_pred)**0.5

0.5876681060137491

In [73]:
mean_squared_error(y_test, y_test_pred)**0.5

0.6715338514796404

### Identifying mismatch cases

* Make a rule based on the predicted value and the actual value that identifies mismatch cases (e.g. difference in actual and predicted being more than a cutoff)

* How many such cases do you see?

* Are all these mismatch cases genuine?

In [74]:
res_df = pd.DataFrame({'review':X_test, 'rating':y_test, 'rating_pred':y_test_pred})

In [75]:
res_df[(res_df.rating - res_df.rating_pred)>=2].shape

(8, 3)

In [76]:
res_df[(res_df.rating - res_df.rating_pred)>=2]

,review,rating,rating_pred
7277,life saviours serving excellent food worst tim...,5.0,2.007670
1818,value money ordered second time,5.0,2.970624
4771,not good,5.0,1.998553
16510,may not polished serving packaging etc never b...,5.0,1.894671
14845,oh memories place first drink bangalore almost...,5.0,2.628792
15201,sauce not included,4.0,1.878766
3165,rice quantity less,5.0,2.844196
16515,may not polished serving packaging etc never b...,5.0,1.894671
